In [15]:
def readDiscussionCSV(inputFilename='ptt.csv', outputFilename='ptt_discussions.pkl', num=None):
    """直接讀取CSV格式的討論串資料，將messages存為留言列表"""
    try:
        # 讀取CSV檔案
        data = pd.read_csv(PATH + inputFilename)
        
        # 如果指定了數量，只取前num筆資料
        if num is not None:
            data = data.head(num)
    except Exception as e:
        print(f"讀取資料失敗: {str(e)}")
        return None
    
    # 檢查必要的欄位是否存在
    required_columns = ['post', 'next_resp']
    if not all(col in data.columns for col in required_columns):
        print(f"資料缺少必要的欄位，需要: {required_columns}")
        return None
    
    # 如果沒有prev_resp欄位，創建一個
    if 'prev_resp' not in data.columns:
        data['prev_resp'] = '<SEP>'
    
    # 按照相同的post分組為討論串
    discussion_threads = []
    current_post = None
    current_messages = []
    thread_index = 0
    
    for idx, row in data.iterrows():
        post = row['post']
        next_resp = row['next_resp']
        prev_resp = row['prev_resp']
        
        # 如果是新的討論主題，開始新的討論串
        if post != current_post:
            if current_messages:
                discussion_threads.append({
                    'thread_id': thread_index,
                    'thread_post': current_post,
                    'messages': current_messages
                })
                thread_index += 1
            current_messages = []
            current_post = post
        
        # 添加當前留言到討論串 - 直接添加留言內容而非字典
        if prev_resp != '<SEP>':
            current_messages.append(prev_resp)
        current_messages.append(next_resp)
    
    # 添加最後一個討論串
    if current_messages:
        discussion_threads.append({
            'thread_id': thread_index,
            'thread_post': current_post,
            'messages': current_messages
        })
    
    # 將討論串資料轉換為DataFrame並保存
    discussions_df = pd.DataFrame({
        'thread_id': [t['thread_id'] for t in discussion_threads],
        'thread_post': [t['thread_post'] for t in discussion_threads],
        'messages_count': [len(t['messages']) for t in discussion_threads],
        'messages': [t['messages'] for t in discussion_threads]
    })
    
    # # 保存處理後的資料
    # discussions_df.to_pickle(PATH + outputFilename)
    # print(f"處理完成: 找到 {len(discussion_threads)} 個討論串")
    
    return discussions_df

In [16]:
discussions_df = readDiscussionCSV(inputFilename='ptt_real.csv', outputFilename='ptt_discussions.pkl', num=None)

In [18]:
print(discussions_df['messages'].iloc[0])

['1. XD XD 他會讀心術 XD', '1. XD XD 他會讀心術 XD', '1.真的很可愛XD 但我覺得他知道你要說什麼啦', '1.真的很可愛XD 但我覺得他知道你要說什麼啦', '火車，尤其是高鐵，不要亂搭', '火車，尤其是高鐵，不要亂搭', '很多人會搞錯以為くまもん就是熊本啊XD', '很多人會搞錯以為くまもん就是熊本啊XD', '重點是有直達鹿耳島的', '重點是有直達鹿耳島的', '第一個好可www好可愛*', '第一個好可www好可愛*', '第一個超好笑  重點旁邊的居然聽得懂XD', '第一個超好笑  重點旁邊的居然聽得懂XD', '幸好他沒有從包包拿出來，那一定超爆笑', '幸好他沒有從包包拿出來，那一定超爆笑', '1. 結果遇到日劇Hero的酒保：あるよ', '1. 結果遇到日劇Hero的酒保：あるよ', '從包包拿出一隻部長：有る\u3000那個畫面XDDDDD', '從包包拿出一隻部長：有る\u3000那個畫面XDDDDD', '大概是猜你要去看熊本熊吧', '大概是猜你要去看熊本熊吧', '第一個蠻好笑的哈哈哈', '第一個蠻好笑的哈哈哈', 'XD覺得好笑', 'XD覺得好笑', '我在餐廳要點蕎麥麵點成炸雞塊完全不會日文', '我在餐廳要點蕎麥麵點成炸雞塊完全不會日文', '你知道kumamon別人還可以猜，我第一次去熊本沒做功課不知道怎麼唸，到處用英文問人說有一隻長的很奇怪的熊在哪裡？結果沒人知道我問什麼...', '你知道kumamon別人還可以猜，我第一次去熊本沒做功課不知道怎麼唸，到處用英文問人說有一隻長的很奇怪的熊在哪裡？結果沒人知道我問什麼...', '長得很奇怪的熊 XDDD部長表示:', '長得很奇怪的熊 XDDD部長表示:', '要講很色的熊(?', '要講很色的熊(?', '問她哦酥酥咩然後就點那個啊', '問她哦酥酥咩然後就點那個啊', '第一個我笑到併軌', '第一個我笑到併軌', '阿魯有XD 好可愛', '阿魯有XD 好可愛', '總之  每道菜來一個', '總之  每道菜來一個', '我笑死']
